In [1]:
import roboticstoolbox as rtb
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import spatialmath as sm

In [2]:
import roboticstoolbox as rtb
robot = rtb.models.Panda()
ets = robot.ets().copy()
print(ets)

SE3(0, 0, 0.333) ⊕ Rz(q0) ⊕ SE3(-90°, -0°, 0°) ⊕ Rz(q1) ⊕ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2) ⊕ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3) ⊕ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) ⊕ SE3(90°, -0°, 0°) ⊕ Rz(q5) ⊕ SE3(0.088, 0, 0; 90°, -0°, 0°) ⊕ Rz(q6) ⊕ SE3(0, 0, 0.107) ⊕ SE3(0°, -0°, -45°) ⊕ SE3(0, 0, 0.1034)


In [3]:
def jacob0(
    ets,
    q,
    tool = None,
):
    r"""
    `\nu = (v_x, v_y, v_z, \omega_x, \omega_y, \omega_z)^T`
    is related to joint velocity by :math:`{}^{E}\!\nu = \mathbf{J}_m(q) \dot{q}`.

    """  # noqa

    # Use c extension
    # try:
    #     return ETS_jacob0(ets._fknm, q, tool)
    # except TypeError:
    #     pass

    # Otherwise use Python
    if tool is None:
        tools = np.eye(4)
    elif isinstance(tool, sm.SE3):
        tools = np.array(tool.A)
    else:  # pragma: nocover
        tools = np.eye(4)

    # q = getvector(q, None)

    T = ets.eval(q, include_base=False) @ tools

    U = np.eye(4)
    j = 0
    J = np.zeros((6, ets.n), dtype=np.float64)
    zero = np.array([0, 0, 0])
    end = ets.data[-1]

    for link in ets.data:

        jindex = 0 if link.jindex is None and link.isjoint else link.jindex

        if link.isjoint:
            U = U @ link.A(q[jindex])  # type: ignore

            if link == end:
                U = U @ tools

            Tu = sm.SE3(U, check=False).inv().A @ T
            n = U[:3, 0]
            o = U[:3, 1]
            a = U[:3, 2]
            x = Tu[0, 3]
            y = Tu[1, 3]
            z = Tu[2, 3]

            if link.axis == "Rz":
                print(f' >> Rz {jindex}<<')
                J[:3, j] = (o * x) - (n * y)
                J[3:, j] = a

            elif link.axis == "Ry":
                J[:3, j] = (n * z) - (a * x)
                J[3:, j] = o

            elif link.axis == "Rx":
                J[:3, j] = (a * y) - (o * z)
                J[3:, j] = n

            elif link.axis == "tx":
                J[:3, j] = n
                J[3:, j] = zero

            elif link.axis == "ty":
                J[:3, j] = o
                J[3:, j] = zero

            elif link.axis == "tz":
                J[:3, j] = a
                J[3:, j] = zero

            j += 1
        else:
            A = link.A()
            if A is not None:
                U = U @ A

    return J

In [4]:
q = ets.random_q()
q = np.zeros(7)
# q = np.ones(7)

In [5]:
# np.set_printoptions(precision=4)
# formatter={'all':lambda x: 'int: '+str(-x)}
# formatter={'all': lambda x: f'{x:.4f}')}
np.set_printoptions(
    edgeitems=3, 
    infstr='inf',
    linewidth=120, 
    nanstr='nan', 
    precision=8,
    suppress=True, 
    threshold=1000, 
    formatter=None
)

J = robot.jacob0(q)
print(type(J), " ", J.dtype)
print(J)
print("\n", robot.fkine(q).A)

<class 'numpy.ndarray'>   float64
[[ 0.      0.4896  0.     -0.1736  0.      0.2104  0.    ]
 [ 0.088  -0.      0.088   0.      0.088   0.      0.    ]
 [-0.     -0.088   0.      0.0055  0.      0.088   0.    ]
 [-0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      1.     -0.     -1.     -0.     -1.     -0.    ]
 [ 1.      0.      1.      0.      1.      0.     -1.    ]]

 [[ 0.70710678  0.70710678  0.          0.088     ]
 [ 0.70710678 -0.70710678 -0.         -0.        ]
 [-0.          0.         -1.          0.8226    ]
 [ 0.          0.          0.          1.        ]]


In [6]:
J = jacob0(ets, q)
print(type(J), " ", J.dtype, " ", type(J[0,0]))
print(J)

print("\n", ets.fkine(q).A)

 >> Rz 0<<
 >> Rz 1<<
 >> Rz 2<<
 >> Rz 3<<
 >> Rz 4<<
 >> Rz 5<<
 >> Rz 6<<
<class 'numpy.ndarray'>   float64   <class 'numpy.float64'>
[[ 0.      0.4896  0.     -0.1736  0.      0.2104  0.    ]
 [ 0.088   0.      0.088   0.      0.088   0.     -0.    ]
 [ 0.     -0.088   0.      0.0055  0.      0.088   0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      1.      0.     -1.      0.     -1.     -0.    ]
 [ 1.      0.      1.      0.      1.      0.     -1.    ]]

 [[ 0.70710678  0.70710678  0.          0.088     ]
 [ 0.70710678 -0.70710678 -0.         -0.        ]
 [-0.          0.         -1.          0.8226    ]
 [ 0.          0.          0.          1.        ]]


In [7]:
T = ets.eval(q, include_base=False)

U = np.eye(4)
j = 0
J = np.zeros((6, ets.n), dtype=np.float64)
zero = np.array([0, 0, 0])
end = ets.data[-1]

for iter_, link in enumerate(ets.data):
    jindex = 0 if link.jindex is None and link.isjoint else link.jindex
    
    if link.isjoint:
        U = U @ link.A(q[jindex])  # type: ignore

        if link == end:
            U = U @ tools
        Tu = sm.SE3(U, check=False).inv().A @ T
        # print(f"Tu = ")
        # print(Tu)
        n = U[:3, 0]
        # print(f"u = {n}")
        o = U[:3, 1]
        a = U[:3, 2]
        x = Tu[0, 3]
        y = Tu[1, 3]
        z = Tu[2, 3]

        if link.axis == "Rz":
            J[:3, j] = (o * x) - (n * y)
            J[3:, j] = a
            # print(f' \t\t\t>> {jindex} << {J[:,j]}')

        elif link.axis == "Ry":
            J[:3, j] = (n * z) - (a * x)
            J[3:, j] = o

        elif link.axis == "Rx":
            J[:3, j] = (a * y) - (o * z)
            J[3:, j] = n

        elif link.axis == "tx":
            J[:3, j] = n
            J[3:, j] = zero

        elif link.axis == "ty":
            J[:3, j] = o
            J[3:, j] = zero

        elif link.axis == "tz":
            J[:3, j] = a
            J[3:, j] = zero

        j += 1
    else:
        A = link.A()
        if A is not None:
            U = U @ A

J

array([[ 0.    ,  0.4896,  0.    , -0.1736,  0.    ,  0.2104,  0.    ],
       [ 0.088 ,  0.    ,  0.088 ,  0.    ,  0.088 ,  0.    , -0.    ],
       [ 0.    , -0.088 ,  0.    ,  0.0055,  0.    ,  0.088 ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  1.    ,  0.    , -1.    ,  0.    , -1.    , -0.    ],
       [ 1.    ,  0.    ,  1.    ,  0.    ,  1.    ,  0.    , -1.    ]])

In [8]:
Jtest = np.array([
    [0.00000000,  0.48960000,  0.00000000, -0.17360000,  0.00000000,  0.21040000,  0.00000000],
    [0.08800000,  0.00000000,  0.08800000,  0.00000000,  0.08800000,  0.00000000, -0.00000000],
    [0.00000000, -0.08800000,  0.00000000,  0.00550000,  0.00000000,  0.08800000,  0.00000000],
    [0.00000000,  0.00000000,  0.00000000,  0.00000000,  0.00000000,  0.00000000,  0.00000000],
    [0.00000000,  1.00000000,  0.00000000, -1.00000000,  0.00000000, -1.00000000, -0.00000000],
    [1.00000000,  0.00000000,  1.00000000,  0.00000000,  1.00000000,  0.00000000, -1.00000000],
])

np.allclose(Jtest, J)

True

In [9]:
def angle_axis_python(T, Td):
    e = np.empty(6)
    e[:3] = Td[:3, -1] - T[:3, -1]
    R = Td[:3, :3] @ T[:3, :3].T
    li = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])
    if base.iszerovec(li):
        # diagonal matrix case
        if np.trace(R) > 0:
            # (1,1,1) case
            a = np.zeros((3,))
        else:
            a = np.pi / 2 * (np.diag(R) + 1)
    else:
        # non-diagonal matrix case
        ln = base.norm(li)
        a = math.atan2(ln, np.trace(R) - 1) * li / ln
    e[3:] = a
    return e

def angle_axis(Te:sm.SE3, Tp:sm.SE3):
    e = np.zeros(6)
    e[3:] = Tp.t - Te.t 
    R = Tp.R * T.R.T 
    li = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])
    R_trace = R.trace()
    li_norm = np.linalg.norm(li)
    if li_norm < 1e-6:
        if R_trace > 0:
            e[:3] = 0
        else:
            e[0] = 0.5 * np.pi * (R[0,0] + 1)
            e[1] = 0.5 * np.pi * (R[1,1] + 1)
            e[2] = 0.5 * np.pi * (R[2,2] + 1)
    else:
        angle = np.arctan2(li_norm, R_trace - 1)
        e[:3] = angle * li / li_norm
    return e

In [10]:
ee_ = np.array([1.7599884037881872, 0.7290110664684424, 1.7599884037881872, 0.262, 0.00000000000000002160276953695931, -0.3725999999999999])
np.linalg.norm(ee_)

2.6332584867367728

In [11]:
# 0.5 * ee_ @ np.eye(6) @ ee_
0.5 * ee_ @ ee_

3.4670251289856187

In [15]:
def ik(
        ets,
        T,
        q0 = None,
        base: sm.SE3 = sm.SE3(),
        tool: sm.SE3 = sm.SE3(),
        maxiter = 50,
        We = np.diag(np.ones(6)),
        tol = 1e-6
):
    qk = ets.random_q()
    if q0 is not None:
        qk = q0.copy()
    #  qk, _ = ik(ets, T, base=base, tool=tool)
    Tk = ets.fkine(qk, base=base)
    np.allclose(Tk.A, T.A, atol=1e-3)
    k = 0
    while k < maxiter:
        k += 1
        Tk = ets.fkine(qk)
        err = angle_axis(base.inv() * Tk, base.inv() * T)
        Err = 0.5 * err @ We @ err
        J = ets.jacob0(qk)
        g = J.T @ We @ err
        λ = 0.1
        Wn = λ * Err * np.eye(ets.n)
        dq = np.linalg.inv(J.T @ We @ J + Wn) @ g
        qk += dq
        if Err < tol:
            break
    qk = np.array([np.arctan2(np.sin(v), np.cos(v)) for v in qk])
    return qk, Err < tol

In [16]:
def ik2(
        ets,
        desired_target,
        q0 = None,
        base: sm.SE3 = sm.SE3(),
        tool: sm.SE3 = sm.SE3(),
        maxiter = 50,
        We = np.diag(np.ones(6)),
        tolerance = 1e-6
):
    qk = ets.random_q()
    if q0 is not None:
        qk = q0.copy()
    #  qk, _ = ik(ets, desired_target, base=base, tool=tool)
    Tk = ets.fkine(qk)
    np.allclose(Tk.A, desired_target.A, atol=1e-3)
    k = 0
    while k < maxiter:
        k += 1
        Tk = ets.fkine(qk)
        residual = rbt.angle_axis(base.inv() * Tk, base.inv() * desired_target)
        residual_norm = 0.5 * residual @ We @ residual
        J = ets.fkine(qk)
        g = J.T @ We @ residual
        ## Gauss-Newton: ===
        # >> q += np.linalg.pinv(J.T @ self.We @ J) @ g + qnull
        #  dq = np.linalg.pinv(J.T @ We @ J) @ g 
        ## Levenberg-Marquardt (LM): ===
        # >> q += np.linalg.inv(J.T @ self.We @ J + Wn) @ g + qnull
        λ = 0.1
        Wn = λ * residual_norm * np.eye(ets.n)
        dq = np.linalg.inv(J.T @ We @ J + Wn) @ g
        qk += dq
        #  logging.logging(f'{k}: (residual_norm={residual_norm})  dq = {dq}')
        if residual_norm < tolerance:
            #  logging.logging(f'EXIT: k = {k}, Error={residual_norm}')
            break
    #  Tk = fk(ets, qk, base=base, tool=tool)
    #  np.allclose(Tk.A, desired_target.A, atol=1e-3)
    qk = np.array([np.arctan2(np.sin(v), np.cos(v)) for v in qk])
    return qk, residual_norm < tolerance

In [17]:
T = sm.SE3.Trans(0.35, 0, 0.45) * sm.SE3.RPY(0, -0.5*np.pi, 0)
# qt, success = ik(ets, T)
qt, success = ik2(ets, T)
ets.fkine(qt)

NameError: name 'fk' is not defined